# Tabular Data

## DataFrames

### Exercise 7
Use the same data as exercise 6, but using a fresh notebook. Perform the following:  

a. Download a table (cross-sectional) containing reference data for each ticker. Reference data should include the full company name and any other useful column data you can
find.  
b. Create a ‘transactions’ table, which has three columns: Date, Ticker, Amount, and BuySell. Add a bunch of transactions (at least ten) of your favorite stocks – both buys and sells (all long positions, no shorts, no day trading). Transactions should span a period of two years. Keep them self-consistent but somewhat arbitrary.  
c. Create a function that takes a date and returns a (cross-sectional) table containing a snapshot of your portfolio at that point in time (Ticker, Position).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; i. The function should join the table with your stock reference data, to enrich with the full company name, etc.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ii. The function should join the table with the stock data table, to retrieve the open price, volume, and return for the given date, for each ticker.  
d. Create a ‘scaffold’ table, which is the concatenation of the above function, for every business date over the two-year period. Be sure to add a ‘Date’ column.
e. ‘PnL’ (Profit and Loss) is your total portfolio profit/loss at any given point in time. For a given stock, the daily marked-to-market PnL is simply the return*position. Add a
column containing the daily PnL for each ticker.  
f. Add a column containing the running cumulative PnL per ticker.  
g. Add a column containing the running cumulative PnL for your entire portfolio.  
h. Create a summary table of your average daily PnL, total PnL, and overall yield per ticker.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; i. ‘Yield’ is PnL/Position.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ii. You should exclude days when you had no position in that stock, from the average.  
i. Display your total PnL at the end of the two years.  

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

#### a. Download a table (cross-sectional) containing reference data for each ticker. Reference data should include the full company name and any other useful column data you can find.

In [41]:
# Reload data from 8.2.6
%store -r data
referenceData = data
referenceData.head(100)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
95,CNC,Centene Corporation,reports,Health Care,Managed Health Care,"St. Louis, Missouri",2016-03-30,1071739,1984
96,CNP,CenterPoint Energy,reports,Utilities,Multi-Utilities,"Houston, Texas",1985-07-31,1130310,1882
97,CERN,Cerner,reports,Health Care,Health Care Technology,"North Kansas City, Missouri",2010-04-30,804753,1979
98,CF,CF Industries Holdings Inc,reports,Materials,Fertilizers & Agricultural Chemicals,"Deerfield, Illinois",2008-08-27,1324404,1946


#### b. Create a ‘transactions’ table, which has three columns: Date, Ticker, Amount, and BuySell. Add a bunch of transactions (at least ten) of your favorite stocks – both buys and sells (all long positions, no shorts, no day trading). Transactions should span a period of two years. Keep them self-consistent but somewhat arbitrary.

In [40]:
import pandas as pd
import datetime

# Adding transaction to the ledger
# param: 
#  date: str (date of the transaction)
#  ticker: str (stock ticker)
#  amount: float (amount of the transaction)
#  buysell: str ('Buy' or 'Sell')
# return:
#  pd.Series (raise exception if buysell is incorrect)
def addTransaction(date, ticker, amount, buysell):
    if not (buysell == 'Buy' or buysell == 'Sell'):
        raise Exception('Invalid buysell parameter, only "Buy" or "Sell"')
    else:
        return pd.Series({'Date': datetime.datetime.strptime(date, '%Y-%m-%d'), 
                          'Ticker': ticker, 
                          'Amount': amount, 
                          'BuySell': buysell
                         }
                        )

########################
# Generate transaction header
transactionsHeader = ['Date', 'Ticker', 'Amount', 'BuySell']

# Init the empty df 
ledger = pd.DataFrame (transactions, columns = transactionsHeader)

# Adding transactions to the ledger
ledger = ledger.append(addTransaction('2020-10-21', 'MMM', 15000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2020-10-27', 'ABT', 120000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2020-11-04', 'MMM', 220000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2020-11-11', 'YUM', 22000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2020-12-21', 'MMM', 323000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2021-01-19', 'AMZN', 722000.00, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2021-02-11', 'AMZN', 21000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2021-06-13', 'AAPL', 920000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2021-08-12', 'ZTS', 75000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2021-10-02', 'YUM', 10000.00, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-01-07', 'ACN', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-02-20', 'SCHW', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-03-30', 'YUM', 5000.00, 'Sell'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-04-01', 'YUM', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-05-09', 'SCHW', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-06-14', 'SCHW', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-07-22', 'MMM', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-11-30', 'AAPL', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-11-30', 'MMM', 20000.00, 'Buy'), ignore_index=True)
ledger = ledger.append(addTransaction('2022-12-11', 'ZTS', 20000.00, 'Buy'), ignore_index=True)

ledger

,Date,Ticker,Amount,BuySell
0,2020-12-21,MMM,15000.0,Buy
1,2020-10-21,ABT,120000.0,Buy
2,2021-07-21,YUM,20000.0,Buy
3,2021-07-21,YUM,22000.0,Buy
4,2021-07-21,YUM,20000.0,Buy
5,2021-07-21,YUM,20000.0,Buy
6,2021-07-21,YUM,20000.0,Buy
7,2021-07-21,YUM,20000.0,Buy
8,2021-07-21,YUM,20000.0,Buy
9,2021-07-21,YUM,20000.0,Buy
